In [ ]:
#Import scipy
import scipy as sci
#Import matplotlib and associated modules for 3D and animations
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
import numpy as np

In [ ]:
#Define universal gravitation constant
G=6.67408e-11 #N-m2/kg2
#Reference quantities
m_nd=5.9722*10**24 #kg #mass of the earth
r_nd=149597870691 # 1 หน่วย ดาราศาสตร์
v_nd=30000 #m/s #relative velocity of earth around the sun
t_nd=365*24*3600 #s #year
#Net constants
K1=G*t_nd*m_nd/(r_nd**2*v_nd)
K2=v_nd*t_nd/r_nd

In [ ]:
n = 1/(3**(0.5))
n**2

In [ ]:
#Define masses
m1=1 #Earth_1
m2=1 #damm earth
m3 = 333000 # sun
#Define initial position vectors
r1=[1,0,0] #m
r2=[n,n,n] #m
r3=[0,0,0] #m
#Convert pos vectors to arrays
r1=np.array(r1,dtype="float64")
r2=np.array(r2,dtype="float64")
r3=np.array(r3,dtype="float64")

#Define initial velocities
v1=[0,0,1] #m/s
v2=[0,0,-1] #m/s
v3=[0,0.01,0]
#Convert velocity vectors to arrays
v1=np.array(v1,dtype="float64")
v2=np.array(v2,dtype="float64")
v3=np.array(v3,dtype="float64")


In [ ]:
#A function defining the equations of motion 
def ThreeBodyEquations(y0,t,K1=K1,K2=K2,m1=m1,m2=m2,m3=m3): 
    r1=y0[:3]
    r2=y0[3:6]
    r3=y0[6:9]
    v1=y0[9:12]
    v2=y0[12:15]
    v3 = y0[15:18]

    
    r12=sci.linalg.norm(r2-r1) #Calculate magnitude or norm of vector
    r13=sci.linalg.norm(r3-r1) #Calculate magnitude or norm of vector
    r23=sci.linalg.norm(r3-r2) #Calculate magnitude or norm of vector

    # ปลาย - ต้น
    dv1bydt=K1*m3*(r3-r1)/r13**3 +K1*m2*(r2-r1)/r12**3
    dv2bydt=K1*m3*(r3-r2)/r23**3 +K1*m1*(r1-r2)/r12**3
    dv3bydt=K1*m1*(r1-r3)/r13**3+K1*m2*(r2-r3)/r23**3

    dr1bydt=K2*v1
    dr2bydt=K2*v2
    dr3bydt=K2*v3

    r_derivs=sci.concatenate((dr1bydt,dr2bydt))
    r_derivs=sci.concatenate((r_derivs,dr3bydt))

    v_derivs=sci.concatenate((dv1bydt,dv2bydt))
    v_derivs=sci.concatenate((v_derivs,dv3bydt))

    derivs=sci.concatenate((r_derivs,v_derivs))
    return derivs

In [ ]:
#Package initial parameters
init_params=sci.array([r1,r2,r3,v1,v2,v3]) #create array of initial params

init_params=init_params.flatten() #flatten array to make it 1D
time_span=sci.linspace(0,20,500) #8 orbital periods and 500 points
#Run the ODE solver
import scipy.integrate
three_body_sol=sci.integrate.odeint(ThreeBodyEquations,init_params,time_span)

In [ ]:
a = three_body_sol[:,:3]
b = three_body_sol[:,3:6]
for i in range(500):
    c = a[i]-b[i]
    print(np.linalg.norm(c))

In [ ]:
r1_sol=three_body_sol[:,:3].T.reshape(3,-1)
r2_sol=three_body_sol[:,3:6].T.reshape(3,-1)
r3_sol=three_body_sol[:,6:9].T.reshape(3,-1)

In [ ]:
r1_sol.shape

In [ ]:
three_body_sol[:,15:18]

## plot

In [ ]:
import numpy as np
%matplotlib notebook
def func(num, dataSet, line):
    # NOTE: there is no .set_data() for 3 dim data...
    line.set_data(dataSet[0:2, :num])    
    line.set_3d_properties(dataSet[2, :num])
    return line
t = r1_sol[2]
x = r1_sol[0]
y = r1_sol[1]
#--------------------------------------------------------------------------------
t2 = r2_sol[2]
x2 = r2_sol[0]
y2 = r2_sol[1]
#------------------------------------------------------------------------------
t3 = r3_sol[2]
x3 = r3_sol[0]
y3 = r3_sol[1]

dataSet1 = np.array([x, y, t])
dataSet2 = np.array([x2, y2, t2])
dataSet3 = np.array([x3, y3, t3])
numdatapoint = len(r1_sol)
fig=plt.figure()
ax = Axes3D(fig)
line1 = ax.plot(r1_sol[0],r1_sol[1],r1_sol[2],color="darkblue",label='Earth')[0]
line2 = ax.plot(r2_sol[0],r2_sol[1],r2_sol[2],color="tab:red",label='Damm Earth')[0]
line3 = ax.plot(r3_sol[0],r3_sol[1],r3_sol[2],color="green",label='Sun')[0]
ax.legend()
interval= 200
line_ani1 = animation.FuncAnimation(fig, func, frames=500, fargs=(dataSet1,line1), interval=interval, blit=True)
line_ani2 = animation.FuncAnimation(fig, func, frames=500, fargs=(dataSet2,line2), interval=interval, blit=True)
line_ani3 = animation.FuncAnimation(fig, func, frames=500, fargs=(dataSet3,line3), interval=interval, blit=True)

#Add a few more bells and whistles
ax.set_xlabel("x-coordinate",fontsize=14)
ax.set_ylabel("y-coordinate",fontsize=14)
ax.set_zlabel("z-coordinate",fontsize=14)
ax.set_title("Visualization of orbits of stars in a two-body system\n",fontsize=14)
